In [1]:
# TODO
#  1. Field
#  2. Embeddings
#  3. Dimensionality

###############################################################################################################

import math
import cmath
import functools
import operator
import numpy as np
import sympy
import mpmath
import scipy
import qutip 
import vpython
import itertools
import random

###############################################################################################################

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

def symmeterize(pieces, labels):
    n = len(pieces)
    unique_labels = list(set(labels))
    label_counts = [0 for i in range(len(unique_labels))]
    label_permutations = itertools.permutations(labels, n)
    for permutation in label_permutations:
        for i in range(len(unique_labels)):
            label_counts[i] += list(permutation).count(unique_labels[i])
    normalization = math.sqrt(functools.reduce(operator.mul, [math.factorial(count) for count in label_counts], 1)/math.factorial(n))    
    normalization = 1./math.sqrt(math.factorial(n))
    permutations = itertools.permutations(pieces, n)
    tensor_sum = sum([qutip.tensor(list(permutation)) for permutation in permutations])
    return normalization*tensor_sum

###############################################################################################################

class Variable:    
    def __init__(self):
        self.value = None
        self.dependencies = []
        self.touched = 0

    def tie(self, other, transformation):
        self.dependencies.append({"other": other, "transformation": transformation})

    def plug(self, value):
        self.value = value
        if self.touched == 0:
            self.touched = 1
        elif self.touched == 1:
            self.touched = 0
        for dependency in self.dependencies:
            other, transformation = dependency["other"], dependency["transformation"]
            if other.touched != self.touched:
                other.plug(transformation(self.value))

    def __str__(self):
        return str(self.value)

###############################################################################################################

class Sphere:
    def __init__(self, x, y, z):
        self.center = [x, y, z]
        self.color = None
        
        self.vsphere = None
        self.vspin_axis = None
        self.vstars = None
        
        self.state = Variable()
        self.vector = Variable()
        self.polynomial = Variable()
        self.roots = Variable()
        self.xyzs = Variable()
        self.angles = Variable()
        self.qubits = Variable()
        self.field = Variable()
        
        self.embeddings = []
        self.embedded_spheres = []
          
        def state_to_vector(state):
            return state.full().T[0]
        self.state.tie(self.vector, state_to_vector)

        def vector_to_state(vector):
            return qutip.Qobj(vector)
        self.vector.tie(self.state, vector_to_state)

        def vector_to_polynomial(vector):
            polynomial = vector.tolist()
            return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))] 
        self.vector.tie(self.polynomial, vector_to_polynomial)

        def polynomial_to_vector(polynomial):
            coordinates = [polynomial[i]/(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) for i in range(len(polynomial))]
            return np.array(coordinates)
        self.polynomial.tie(self.vector, polynomial_to_vector)

        def polynomial_to_roots(polynomial):
            try:
                return [np.conjugate(complex(root)) for root in mpmath.polyroots(polynomial)]
            except:
                return [complex(0,0) for i in range(len(polynomial)-1)]
        self.polynomial.tie(self.roots, polynomial_to_roots)

        def roots_to_polynomial(roots):
            s = sympy.symbols("s")
            polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s-np.conjugate(root) for root in roots]), domain="CC")
            return [complex(c) for c in polynomial.coeffs()]
        self.roots.tie(self.polynomial, roots_to_polynomial)

        def roots_to_xyzs(roots):
            def root_to_xyz(root):
                if root == float('inf'):
                    return [0,0,1]
                x = root.real
                y = root.imag
                return [(2*x)/(1.+(x**2)+(y**2)),\
                        (2*y)/(1.+(x**2)+(y**2)),\
                        (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]
            return [root_to_xyz(root) for root in roots]
        self.roots.tie(self.xyzs, roots_to_xyzs)

        def xyzs_to_roots(xyzs):
            def xyz_to_root(xyz):
                x, y, z = xyz[0], xyz[1], xyz[2]
                if z == 1:
                    return float('inf') 
                else:
                    return complex(x/(1-z), y/(1-z))
            return [xyz_to_root(xyz) for xyz in xyzs]
        self.xyzs.tie(self.roots, xyzs_to_roots)
        
        # theta: colatitude with respect to z-axis
        # phi: longitude with respect to y-axis
                
        def xyzs_to_angles(xyzs):
            def xyz_to_angle(xyz):
                x, y, z = xyz[0], -1*xyz[1], -1*xyz[2]
                r = math.sqrt(x**2 + y**2 + z**2)
                latitude = math.asin(z/r)
                colatitude = (math.pi/2.)-latitude
                longitude = None
                if x > 0:
                    longitude = math.atan(y/x)
                elif y > 0:
                    longitude = math.atan(y/x) + math.pi
                else:
                    longitude = math.atan(y/x) - math.pi
                return (colatitude, longitude)
            return [xyz_to_angle(xyz) for xyz in xyzs]
        self.xyzs.tie(self.angles, xyzs_to_angles)
        
        def angles_to_xyzs(angles):
            def angle_to_xyz(angle):
                theta, phi = angle
                return [math.sin(theta)*math.cos(phi),\
                        -1*math.sin(theta)*math.sin(phi),\
                        -1*math.cos(theta)]
            return [angle_to_xyz(angle) for angle in angles]
        self.angles.tie(self.xyzs, angles_to_xyzs)
        
        def angles_to_qubits(angles):
            def angle_to_qubit(angle):
                theta, phi = angle
                qubit = [math.cos(theta/2.), math.sin(theta/2.)*cmath.exp(complex(0,1)*phi)]
                return qutip.Qobj(np.array(qubit))
            return [angle_to_qubit(angle) for angle in angles]
        self.angles.tie(self.qubits, angles_to_qubits)
        
        def qubits_to_angles(qubits):
            def qubit_to_angle(qubit):
                alpha, beta = qubit.full().T[0].tolist()
                alpha_r, alpha_theta = cmath.polar(alpha)
                beta_r, beta_theta = cmath.polar(beta)      
                theta = 2*math.acos(abs(alpha_r))
                phi = beta_theta-alpha_theta
                return (theta, phi)
            return [qubit_to_angle(qubit) for qubit in qubits]
        self.qubits.tie(self.angles, qubits_to_angles)
        
        def qubits_to_field(qubits):
            field = {"particles": [i for i in range(len(qubits))],\
                     "normalization": 1}
            return field
        self.qubits.tie(self.field, qubits_to_field)
        
        def field_to_qubits(field):
            particles = field["particles"]
            normalization = field["normalization"]
            return [self.qubits.value[particle] for particle in particles]
        self.field.tie(self.qubits, field_to_qubits)
        
    def visualize(self):
        if self.state.value != None:
            if self.vsphere == None:
                self.vsphere = vpython.sphere()
            if self.vspin_axis == None:
                self.vspin_axis = vpython.arrow()
            if self.vstars == None:
                self.vstars = [vpython.sphere() for i in range(len(self.xyzs.value))]
            vpython.rate(100)
            self.vsphere.pos = vpython.vector(*self.center)
            self.vsphere.radius = np.linalg.norm(np.array(self.spin_axis()))
            self.vsphere.color = self.color
            self.vsphere.opacity = 0.4
            self.vspin_axis.pos = vpython.vector(*self.center)
            self.vspin_axis.axis = vpython.vector(*self.spin_axis())
            self.vspin_axis.color = self.color
            for i in range(len(self.vstars)):
                self.vstars[i].pos = self.vsphere.pos + self.vsphere.radius*vpython.vector(*self.xyzs.value[i])
                self.vstars[i].radius = 0.1*self.vsphere.radius
                self.vstars[i].color = vpython.color.white
                self.vstars[i].opacity = 0.8
            
            for i in range(len(self.embeddings)):
                my_projector = self.state.value.ptrace(0)
                other_state = self.embeddings[i].state.value
                projected_state = my_projector*other_state
                self.embedded_spheres[i].center = self.center
                self.embedded_spheres[i].state.plug(projected_state)
                self.embedded_spheres[i].visualize()
    
    def spin_operators(self):
        if self.state.value != None:
            n = len(self.vector.value)
            spin = (n-1.)/2.
            return {"X": qutip.jmat(spin, "x"),\
                    "Y": qutip.jmat(spin, "y"),\
                    "Z": qutip.jmat(spin, "z"),\
                    "+": qutip.jmat(spin, "+"),\
                    "-": qutip.jmat(spin, "-")}
    
    def qubit_operators(self):
        return {"X": qutip.jmat(0.5, "x"),\
                "Y": qutip.jmat(0.5, "y"),\
                "Z": qutip.jmat(0.5, "z"),\
                "+": qutip.jmat(0.5, "+"),\
                "-": qutip.jmat(0.5, "-")}
    
    def spin_axis(self):
        spin_ops = self.spin_operators()
        return [-1*qutip.expect(spin_ops["X"], self.state.value),\
                qutip.expect(spin_ops["Y"], self.state.value),\
                qutip.expect(spin_ops["Z"], self.state.value)]
    
    def evolve(self, hermitian, inverse=False, dt=0.007):
        unitary = qutip.Qobj(scipy.linalg.expm(-2*math.pi*complex(0,1)*hermitian.full()*dt))
        if inverse:
            unitary = unitary.dag()
        self.state.plug(unitary*self.state.value)
        
    def evolve_qubit(self, i, hermitian, inverse=False, dt=0.007):
        unitary = qutip.Qobj(scipy.linalg.expm(-2*math.pi*complex(0,1)*hermitian.full()*dt))
        if inverse:
            unitary = unitary.dag()
        qubits = self.qubits.value[:]
        qubits[i] = unitary*qubits[i]
        self.qubits.plug(qubits)
        
    def creation_operators(self):
        a = qutip.Qobj(np.array([[0,0],[1,0]]))
        b = qutip.Qobj(np.array([[1,0],[0,0]]))
        creation_operators = []
        for qubit in self.qubits.value:
            alpha, beta = qubit.full().T[0].tolist()
            creation_operator = alpha*a + beta*b
            creation_operators.append(creation_operator)
        return creation_operators

    def annihilation_operators(self):
        return [op.dag() for op in self.creation_operators()]
    
    def create(self, i):
        field = self.field.value
        normalization = field["normalization"]
        particles = field["particles"]
        normalization *= math.sqrt(len(particles)+1)
        particles.append(i)
        field["normalization"] = normalization
        field["particles"] = particles
        self.vstars.append(vpython.sphere())
        self.field.plug(field)
    
    def destroy(self, i):
        if len(self.qubits.value) > 1:
            field = self.field.value
            normalization = field["normalization"]
            particles = field["particles"]
            normalization *= math.sqrt(len(particles))*\
                             self.qubits.value[i].overlap(self.qubits.value[particles[-1]])
            del particles[-1]
            field["normalization"] = normalization
            field["particles"] = particles
            self.field.plug(field)
            self.vstars[-1].visible = False
            del self.vstars[-1]
    
    def field_state(self):
        field = self.field.value
        normalization = field["normalization"]
        particles = field["particles"]
        return normalization*symmeterize([self.qubits.value[i] for i in particles], particles)
    
    def embed(self, other):
        self.embeddings.append(other)
        embedded_sphere = Sphere(*self.center)
        embedded_sphere.color = other.color
        self.embedded_spheres.append(embedded_sphere)
        
    def unembed(self, other):
        i = self.embeddings.index(other)
        self.embeddings.remove(other)
        self.embedded_spheres[i].kill()
        
    def kill(self):
        if self.vsphere:
            self.vsphere.visible = False
            del self.vsphere
            self.vsphere = None
        if self.vspin_axis:
            self.vspin_axis.visible = False
            del self.vspin_axis
            self.vspin_axis = None
        if self.vstars:
            for vstar in self.vstars:
                vstar.visible = False
                del vstar
            self.vstars = None
        if self.embedded_spheres:
            for sphere in self.embedded_spheres:
                sphere.kill()
        
###############################################################################################################

colors = [vpython.color.red, vpython.color.green, vpython.color.blue,\
          vpython.color.yellow, vpython.color.orange, vpython.color.cyan,\
          vpython.color.magenta]
n = 1
d = 2
mutual_embedding = False

spheres = [Sphere(0,0,0) for i in range(n)]

sphere_colors = random.sample(colors, n)
for i in range(n):
    spheres[i].color = sphere_colors[i]

if mutual_embedding:
    for i in range(n):
        for j in range(n):
            if i != j:
                spheres[i].embed(spheres[j])

###############################################################################################################

vpython.scene.width = 600
vpython.scene.height = 800
vpython.scene.userspin = True

selected = None
sphere_selected = 0

def mouse(event):
    global selected
    global sphere_selected
    selected = vpython.scene.mouse.pick
    for sphere in spheres:
        if sphere.vsphere == selected:
            sphere_selected = spheres.index(sphere)
vpython.scene.bind('click', mouse)

def keyboard(event):
    global sphere
    global selected
    global sphere_selected
    key = event.key
    spin_ops = spheres[sphere_selected].spin_operators()
    qubit_ops = spheres[sphere_selected].qubit_operators()
    if key == "a":  
        spheres[sphere_selected].evolve(spin_ops["X"], inverse=True)
    elif key == "d": 
        spheres[sphere_selected].evolve(spin_ops["X"], inverse=False)
    elif key == "s": 
        spheres[sphere_selected].evolve(spin_ops["Z"], inverse=True)
    elif key == "w": 
        spheres[sphere_selected].evolve(spin_ops["Z"], inverse=False)
    elif key == "z": 
        spheres[sphere_selected].evolve(spin_ops["Y"], inverse=True)
    elif key == "x": 
        spheres[sphere_selected].evolve(spin_ops["Y"], inverse=False)
    elif key == "q": 
        spheres[sphere_selected].evolve(spin_ops["-"], inverse=False)
    elif key == "1":
        spheres[sphere_selected].evolve(spin_ops["-"], inverse=True)
    elif key == "e":
        spheres[sphere_selected].evolve(spin_ops["+"], inverse=False)
    elif key == "3":
        spheres[sphere_selected].evolve(spin_ops["+"], inverse=True)   
    elif key == "j":
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["X"], inverse=True)
    elif key == "l": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["X"], inverse=False)
    elif key == "k": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["Z"], inverse=True)
    elif key == "i": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["Z"], inverse=False)
    elif key == "m": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["Y"], inverse=True)
    elif key == ",": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["Y"], inverse=False)
    elif key == "u": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["-"], inverse=False)
    elif key == "7": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["-"], inverse=True)
    elif key == "o": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["+"], inverse=False)
    elif key == "0": 
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].evolve_qubit(i, qubit_ops["+"], inverse=True)
    elif key == "+":
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].create(i)
    elif key == "-":
        if selected and selected in spheres[sphere_selected].vstars:
            i = spheres[sphere_selected].vstars.index(selected)
            spheres[sphere_selected].destroy(i)    
    elif key == "`":
        field_state = spheres[sphere_selected].field_state()
        print(field_state)
vpython.scene.bind('keydown', keyboard)

###############################################################################################################

for i in range(n):
    spheres[i].state.plug(qutip.rand_ket(d))

for sphere in spheres:
    sphere.visualize()
        
vpython.scene.camera.follow(spheres[int(n/2)].vsphere)

while True:
    account = 0
    for i in range(n):
        if i == 0:
            spheres[i].center = [0,0,0]
        elif i > 0:
            account += spheres[i].vsphere.radius+spheres[i-1].vsphere.radius
            spheres[i].center = [0, account, 0]
        spheres[i].visualize()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TypeError: Incompatible Qobj shapes